In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import os 
import numpy as np 
import pandas as pd
from sklearn import preprocessing 
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier

In [3]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/autoencoder_features/data.csv"
df = pd.read_csv(path)
df.head() 

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,Level 5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,Level 5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,Level 5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",Level 5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Level 2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [4]:
df.iloc[:,1].replace(['Level 1', 'Level 2', 'Level 3', 'Level 4', 'Level 5'],
                        [1, 2, 3, 4, 5], inplace=True)

In [5]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [6]:
#define bins
bins = [0,2,4,5]
#name of the group 
group_names = ['Low', 'Moderate', 'High']
df['Level '] = pd.cut(df["Level "],bins,labels = group_names)

In [7]:
df.head(5)

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,High,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,High,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,High,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",High,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Low,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [8]:
df['Level '] = df['Level '].map({'Low': 1, 'Moderate': 2, 'High': 3}).astype(int)

In [9]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,3,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,3,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,3,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",3,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,1,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [10]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/autoencoder_features/fasttext_2autoencode_withstopwordchangelevels_20_features.csv"
x = pd.read_csv(path)
x.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,10,11,12,13,14,15,16,17,18,19
0,0,0.018019,-0.029538,-0.055239,0.049131,0.042634,-0.055844,-0.013113,-0.028185,0.063799,...,0.042719,-0.024424,0.021971,-0.031706,0.045135,-0.046555,-0.059222,-0.004283,-0.001129,-0.056292
1,1,-0.030832,-0.042466,0.014082,0.039814,0.049848,-0.004768,-0.033239,0.011525,-0.005728,...,-0.040244,-0.024236,0.005437,-0.035944,-0.050629,-0.032357,-0.025965,-0.021170,0.011179,0.008437
2,2,-0.007108,-0.018637,-0.028971,0.021180,0.015200,-0.022956,-0.030151,0.022720,-0.003780,...,0.020665,-0.031942,0.012797,-0.034789,-0.003919,-0.047922,0.044918,0.033416,-0.013371,0.038272
3,3,-0.042697,0.001645,0.036530,0.051700,0.055478,0.056080,-0.019043,-0.031062,-0.011803,...,-0.047197,0.057927,-0.036228,0.050132,0.057844,-0.019156,0.009239,-0.006890,-0.030865,0.004539
4,4,-0.060208,0.016306,-0.007716,0.050473,-0.017961,0.019145,-0.030577,-0.031559,-0.037713,...,-0.006990,-0.049243,0.024274,-0.014497,-0.019525,0.019573,0.033455,-0.015994,-0.017160,0.016144


In [11]:
x.drop(x.columns[[0]], axis=1, inplace=True)
x.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.018019,-0.029538,-0.055239,0.049131,0.042634,-0.055844,-0.013113,-0.028185,0.063799,-0.032372,0.042719,-0.024424,0.021971,-0.031706,0.045135,-0.046555,-0.059222,-0.004283,-0.001129,-0.056292
1,-0.030832,-0.042466,0.014082,0.039814,0.049848,-0.004768,-0.033239,0.011525,-0.005728,0.018363,-0.040244,-0.024236,0.005437,-0.035944,-0.050629,-0.032357,-0.025965,-0.021170,0.011179,0.008437
2,-0.007108,-0.018637,-0.028971,0.021180,0.015200,-0.022956,-0.030151,0.022720,-0.003780,0.021422,0.020665,-0.031942,0.012797,-0.034789,-0.003919,-0.047922,0.044918,0.033416,-0.013371,0.038272
3,-0.042697,0.001645,0.036530,0.051700,0.055478,0.056080,-0.019043,-0.031062,-0.011803,0.004662,-0.047197,0.057927,-0.036228,0.050132,0.057844,-0.019156,0.009239,-0.006890,-0.030865,0.004539
4,-0.060208,0.016306,-0.007716,0.050473,-0.017961,0.019145,-0.030577,-0.031559,-0.037713,0.015892,-0.006990,-0.049243,0.024274,-0.014497,-0.019525,0.019573,0.033455,-0.015994,-0.017160,0.016144


In [12]:
y=df['Level '] 

**Train Test Split**

In [13]:
# Train-Test-Split is performed

from sklearn.model_selection import train_test_split

# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=42)
X_train.shape, X_test.shape

((1220, 20), (524, 20))

In [14]:
print(y_train.value_counts(normalize=True)*100)
print("\n")
print(y_test.value_counts(normalize=True)*100)

2    45.327869
1    29.590164
3    25.081967
Name: Level , dtype: float64


2    45.229008
1    29.961832
3    24.809160
Name: Level , dtype: float64


**Model Building**

In [ ]:
def perceptron_classifier (X_train, y_train, X_test, y_test):
  p = Perceptron(random_state=150)
  p.fit(X_train, y_train)
  predictions_train = p.predict(X_train)
  predictions_test = p.predict(X_test)
  train_score = accuracy_score(predictions_train, y_train)
  #print("score on train data: ", train_score)
  test_score = f1_score(predictions_test, y_test, average='weighted' )
  print("Perceptron F1 score:", test_score)

In [ ]:
from sklearn.linear_model import Perceptron
perceptron_classifier(X_train, y_train, X_test, y_test) 

Perceptron F1 score: 0.31233329474253874


**Decision Tree**

In [15]:
clf_dt = DecisionTreeClassifier(max_depth=5, class_weight = "balanced")

In [16]:
clf_dt.fit(X_train,y_train)

DecisionTreeClassifier(class_weight='balanced', max_depth=5)

In [17]:
y_pred_train = clf_dt.predict(X_train)
y_pred_test = clf_dt.predict(X_test)

In [18]:
def classifcation_report_train_test(y_train, y_train_pred, y_test, y_test_pred):

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TRAIN DATA
            =========================================
            ''')
    print(classification_report(y_train, y_train_pred, digits=4))

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TEST DATA
            =========================================
            ''')
    print(classification_report(y_test, y_test_pred, digits=4))

In [19]:
scores = pd.DataFrame(columns=['Model', 'Train_Accuracy', 'Train_Recall', 'Train_Precision', 'Train_F1_Score', 
                               'Test_Accuracy', 'Test_Recall', 'Test_Precision', 'Test_F1_Score'])

def get_metrics(train_actual, train_predicted, test_actual, test_predicted, model_description, dataframe):

    train_accuracy  = accuracy_score(train_actual, train_predicted)
    train_recall    = recall_score(train_actual, train_predicted, average="weighted")
    train_precision = precision_score(train_actual, train_predicted, average="weighted")
    train_f1score   = f1_score(train_actual, train_predicted, average="weighted")
    
    test_accuracy   = accuracy_score(test_actual, test_predicted)
    test_recall     = recall_score(test_actual, test_predicted, average="weighted")
    test_precision  = precision_score(test_actual, test_predicted, average="weighted")
    test_f1score    = f1_score(test_actual, test_predicted, average="weighted")

    dataframe       = dataframe.append(pd.Series([model_description, 
                                                  train_accuracy, train_recall, train_precision, train_f1score,
                                                  test_accuracy, test_recall, test_precision, test_f1score],
                                                 index=scores.columns ), 
                                       ignore_index=True)

    return(dataframe)

In [20]:
scores = get_metrics(y_train, y_pred_train, y_test, y_pred_test, "DecisionTrees", scores)
scores

<ipython-input-19-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.485246,0.485246,0.518762,0.451103,0.414122,0.414122,0.382368,0.365769


**Random Forest**

In [21]:
clf_rf = RandomForestClassifier(class_weight = "balanced")

In [22]:
clf_rf.fit(X=X_train, y=y_train)

RandomForestClassifier(class_weight='balanced')

In [23]:
y_pred_train1 = clf_rf.predict(X_train)
y_pred_test1 = clf_rf.predict(X_test)

In [24]:
classifcation_report_train_test(y_train, y_pred_train1, y_test, y_pred_test1)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     1.0000    1.0000    1.0000       361
           2     1.0000    1.0000    1.0000       553
           3     1.0000    1.0000    1.0000       306

    accuracy                         1.0000      1220
   macro avg     1.0000    1.0000    1.0000      1220
weighted avg     1.0000    1.0000    1.0000      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2909    0.1019    0.1509       157
           2     0.4503    0.8608    0.5913       237
           3     0.3125    0.0385    0.0685       130

    accuracy                         0.4294       524
   macro avg     0.3512    0.3337    0.2702       524
weighted avg     0.3684    0.4294    0.3297       524



In [25]:
scores = get_metrics(y_train, y_pred_train1, y_test, y_pred_test1, "RandomForest", scores)
scores

<ipython-input-19-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.485246,0.485246,0.518762,0.451103,0.414122,0.414122,0.382368,0.365769
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.429389,0.429389,0.368371,0.329659


**AdaBoost**

In [ ]:
clf_adaboost =  AdaBoostClassifier()

In [ ]:
clf_adaboost.fit(X_train, y_train)

AdaBoostClassifier()

In [ ]:
y_pred_train2 = clf_adaboost.predict(X_train)
y_pred_test2 = clf_adaboost.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train2, y_test, y_pred_test2)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.5494    0.3850    0.4528       361
           2     0.5160    0.7884    0.6237       553
           3     0.5082    0.2026    0.2897       306

    accuracy                         0.5221      1220
   macro avg     0.5245    0.4587    0.4554      1220
weighted avg     0.5239    0.5221    0.4894      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3011    0.1783    0.2240       157
           2     0.4508    0.6962    0.5473       237
           3     0.2462    0.1231    0.1641       130

    accuracy                         0.3989       524
   macro avg     0.3327    0.3325    0.3118       524
weighted avg     0.3552    0.3989    0.3553       524



In [ ]:
scores = get_metrics(y_train, y_pred_train2, y_test, y_pred_test2, "Adaboost", scores)
scores

<ipython-input-21-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.509836,0.509836,0.566639,0.450641,0.419847,0.419847,0.365550,0.352990
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.440840,0.440840,0.412868,0.360249
2,Adaboost,0.522131,0.522131,0.523917,0.489374,0.398855,0.398855,0.355178,0.355349


**Gradient Boosting Classifier**

In [ ]:
clf_gbm = GradientBoostingClassifier()

In [ ]:
clf_gbm.fit(X=X_train, y=y_train)

GradientBoostingClassifier()

In [ ]:
y_pred_train3 = clf_gbm.predict(X_train)
y_pred_test3 = clf_gbm.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train3, y_test, y_pred_test3)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.9579    0.6925    0.8039       361
           2     0.7279    0.9964    0.8412       553
           3     0.9901    0.6536    0.7874       306

    accuracy                         0.8205      1220
   macro avg     0.8919    0.7808    0.8108      1220
weighted avg     0.8617    0.8205    0.8167      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2473    0.1465    0.1840       157
           2     0.4444    0.7257    0.5513       237
           3     0.2500    0.0846    0.1264       130

    accuracy                         0.3931       524
   macro avg     0.3139    0.3190    0.2872       524
weighted avg     0.3371    0.3931    0.3358       524



In [ ]:
scores = get_metrics(y_train, y_pred_train3, y_test, y_pred_test3, "GBM", scores)
scores

<ipython-input-21-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.509836,0.509836,0.566639,0.450641,0.419847,0.419847,0.365550,0.352990
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.440840,0.440840,0.412868,0.360249
2,Adaboost,0.522131,0.522131,0.523917,0.489374,0.398855,0.398855,0.355178,0.355349
3,GBM,0.820492,0.820492,0.861696,0.816667,0.393130,0.393130,0.337140,0.335837


**SVM Model**

In [26]:
linear_svm = SVC(kernel='linear', C=1, class_weight = "balanced")

In [27]:
linear_svm.fit(X=X_train, y= y_train)

SVC(C=1, class_weight='balanced', kernel='linear')

In [28]:
y_pred_train4 = linear_svm.predict(X_train)
y_pred_test4 = linear_svm.predict(X_test)

In [29]:
classifcation_report_train_test(y_train, y_pred_train4, y_test, y_pred_test4)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.0000    0.0000    0.0000       361
           2     0.0000    0.0000    0.0000       553
           3     0.2508    1.0000    0.4010       306

    accuracy                         0.2508      1220
   macro avg     0.0836    0.3333    0.1337      1220
weighted avg     0.0629    0.2508    0.1006      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.0000    0.0000    0.0000       157
           2     0.0000    0.0000    0.0000       237
           3     0.2481    1.0000    0.3976       130

    accuracy                         0.2481       524
   macro avg     0.0827    0.3333    0.1325       524
weighted avg     0.0615    0.2481    0.0986       524



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

In [30]:
scores = get_metrics(y_train, y_pred_train4, y_test, y_pred_test4, "SVM", scores)
scores

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
<ipython-input-19-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.485246,0.485246,0.518762,0.451103,0.414122,0.414122,0.382368,0.365769
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.429389,0.429389,0.368371,0.329659
2,SVM,0.250820,0.250820,0.062911,0.100591,0.248092,0.248092,0.061549,0.098630


**KNN**

In [ ]:
knn = KNeighborsClassifier(n_neighbors=7)

In [ ]:
knn.fit(X=X_train, y=y_train)

KNeighborsClassifier(n_neighbors=7)

In [ ]:
y_pred_train5 = knn.predict(X_train)
y_pred_test5 = knn.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train5, y_test, y_pred_test5)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.4738    0.5263    0.4987       361
           2     0.5899    0.7414    0.6571       553
           3     0.5403    0.2190    0.3116       306

    accuracy                         0.5467      1220
   macro avg     0.5347    0.4956    0.4891      1220
weighted avg     0.5431    0.5467    0.5236      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2903    0.2866    0.2885       157
           2     0.4649    0.5865    0.5187       237
           3     0.3571    0.1923    0.2500       130

    accuracy                         0.3989       524
   macro avg     0.3708    0.3551    0.3524       524
weighted avg     0.3859    0.3989    0.3830       524



In [ ]:
scores = get_metrics(y_train, y_pred_train5, y_test, y_pred_test5, "KNN", scores)
scores

<ipython-input-21-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.509836,0.509836,0.566639,0.450641,0.419847,0.419847,0.365550,0.352990
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.440840,0.440840,0.412868,0.360249
2,Adaboost,0.522131,0.522131,0.523917,0.489374,0.398855,0.398855,0.355178,0.355349
3,GBM,0.820492,0.820492,0.861696,0.816667,0.393130,0.393130,0.337140,0.335837
4,SVM,0.453279,0.453279,0.205462,0.282756,0.452290,0.452290,0.204566,0.281716
5,KNN,0.546721,0.546721,0.543128,0.523552,0.398855,0.398855,0.385852,0.383035


**GNB**

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()

In [ ]:
nb.fit(X=X_train, y=y_train)

GaussianNB()

In [ ]:
y_pred_train6 = nb.predict(X_train)
y_pred_test6 = nb.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train6, y_test, y_pred_test6)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.4583    0.1219    0.1926       361
           2     0.4641    0.9349    0.6203       553
           3     0.6000    0.0196    0.0380       306

    accuracy                         0.4648      1220
   macro avg     0.5075    0.3588    0.2836      1220
weighted avg     0.4965    0.4648    0.3477      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3939    0.0828    0.1368       157
           2     0.4517    0.9283    0.6077       237
           3     0.0000    0.0000    0.0000       130

    accuracy                         0.4447       524
   macro avg     0.2819    0.3370    0.2482       524
weighted avg     0.3224    0.4447    0.3159       524



In [ ]:
scores = get_metrics(y_train, y_pred_train6, y_test, y_pred_test6, "GNB", scores)
scores

<ipython-input-21-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.509836,0.509836,0.566639,0.450641,0.419847,0.419847,0.365550,0.352990
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.440840,0.440840,0.412868,0.360249
2,Adaboost,0.522131,0.522131,0.523917,0.489374,0.398855,0.398855,0.355178,0.355349
3,GBM,0.820492,0.820492,0.861696,0.816667,0.393130,0.393130,0.337140,0.335837
4,SVM,0.453279,0.453279,0.205462,0.282756,0.452290,0.452290,0.204566,0.281716
5,KNN,0.546721,0.546721,0.543128,0.523552,0.398855,0.398855,0.385852,0.383035
6,GNB,0.464754,0.464754,0.496477,0.347662,0.444656,0.444656,0.322351,0.315873
